# Variational Quantum Eigensolver

This is a method used to find the quantum states of a quantum system using variational methods. Such as those used in analytical mechanics when you are minimizing the variation of an action. In this case we are going to use this method to find the ground state energy of a molecule following the [tutorial](https://pennylane.ai/qml/demos/tutorial_vqe.html) offered by Xanadú using PennyLane.

In this example we are going to find the ground state wave function of the $H_2$ hydrogen molecule. This is going to be done starting from the ansatz $|\Psi\rangle = \alpha|1100\rangle + \beta|0011\rangle$. The obtention of this function is not clear for me at this moment. What I know is that this state will conserve certain type of antisymmetry since the two electrons involved must be represented by an antisymetrical function. This quantum state contemplates the spin state of the electrons and the orbitals that they are occupying, I think that is the reason why there are 4 quantum numbers.

Now that we have an ansatz PennyLane is going to optimize the parameters $\alpha$ and $\beta$ in order to minimize the energy of the ground stated.


## Build the electronic Hamiltonian

We import some libraries

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

Now we have to specify the properties of the molecular state that we want to calculate. For this we have to provide three thing:
- The geometry
    This implies that we have to specify the coordinates in space of the atoms that compose the molecule of interest. This can be done using an special type of file for chemistry with extension `.xyz`. This file contains a table of the coordinates of the molecule's atoms. There are several databases of this files. Personally I liked [SMART-SNS](http://smart.sns.it/molecules/) because it shows you the position of the atoms in space and you can easily rotate the structure in order to visualize the molecule.
    
- The charge
    This charge represents the (integer) number of electrons that where added or removed from the neutral molecule. This could vary but, in most of the cases, the charge number will be 0.
    
- The multiplicity
    Corresponds to the multiplicity of a degenerated energy level due to the spin states of the unpaired electrons in the molecule. It is defined in terms of the total spin angular momentum $S$ as $M = 2S + 1$. This represents all the possible spin states of the molecule, then, these states are degenerated states because they have the same energy. In this case all electrons are paired so the spin of these electrons is $0$, then, multiplicity is $1$.


In [2]:
## Import the geometry file
geometry = "VQE_files/h2.xyz"

## Define the charge of unpaired electrons
charge = 0

## Define the multiplicity
multiplicity = 1

There is an additional information that we have to provide to the algorithm. We have to specify the basis set used to approximate the atomic orbitals. This is a set of function that we can use to represent the wave function of the electrons in orther to turn the differential equation of the model into an algebraic equation using the Hartree-Fock approximation. In this case we are using the minimal basis STO-3g where a set of 3 gaussians (3g) represent an atomic Slater-type orbital (STO). That is why is called STO-3g.

In [3]:
basis_set = 'sto-3g'

Now we have to compute the Hamiltonian of the molecule in the Pauli basis. This can be done using the PennyLane function `molecular_hamiltonian()`. The arguments of this function are the parameters that we have already specify with the additional information of the fermionic-to-qubit mapping. This maps quantum creation and annihilation operators into computational operators. For this example we are using the Jordan-Wigner transformation.

In [5]:
## Here we extract the information of the xyz file
symbols, coordinates = qchem.read_structure(geometry)

## Now we compute the Hamiltonian and the ammount of qubits needed to simulate the molecule
h, qubits = qchem.molecular_hamiltonian(
symbols,
coordinates,
charge = charge,
mult = multiplicity,
basis = basis_set,
active_electrons = 2,
active_orbitals = 2,
mapping = 'jordan_wigner')

print('Number of qubits = ',qubits)
print('Hamiltonian is ',h)

Number of qubits =  4
Hamiltonian is  (-0.04207897647782277) [I0]
+ (0.17771287465139946) [Z0]
+ (0.1777128746513994) [Z1]
+ (-0.24274280513140462) [Z2]
+ (-0.24274280513140462) [Z3]
+ (0.17059738328801052) [Z0 Z1]
+ (0.04475014401535161) [Y0 X1 X2 Y3]
+ (-0.04475014401535161) [Y0 Y1 X2 X3]
+ (-0.04475014401535161) [X0 X1 Y2 Y3]
+ (0.04475014401535161) [X0 Y1 Y2 X3]
+ (0.12293305056183801) [Z0 Z2]
+ (0.1676831945771896) [Z0 Z3]
+ (0.1676831945771896) [Z1 Z2]
+ (0.12293305056183801) [Z1 Z3]
+ (0.1762764080431959) [Z2 Z3]


## Implementing the VQE algorithm
Now we are ready to implement the algorithm. PennyLane has a class called `Expvalcost`. This class is used to build up the cost function for a given Hamiltonian taking into account the circuit and the device in which the algorithm is going to be excecuted 

In [8]:
## First we define the device
dev = qml.device('default.qubit', wires=qubits)

In [10]:
## Now we define de circuit of the problem
def circuit(params, wires):
    qml.BasisState(np.array([1, 1, 0, 0], requires_grad=False), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])

Here we use the `ExpvalCost` class to find the cost function of this problem.

In [11]:
cost_fn = qml.ExpvalCost(circuit,h,dev)

Here we define some random parameters to initialize the circuit

In [12]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
np.random.seed(0)
params = np.random.normal(0,np.pi,(qubits, 3))

print(params)

[[ 5.54193389  1.25713095  3.07479606]
 [ 7.03997361  5.86710646 -3.07020901]
 [ 2.98479079 -0.47550269 -0.32427159]
 [ 1.28993324  0.45252622  4.56873497]]


We define a maximum of 200 steps to reach a convergence tolerance of $\approx 10^{-6}$

In [13]:
max_iterations = 200
conv_tol = 1e-06

for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)
    
    if n%20 == 0:
        print('iteration = {:}, Energy = {:.8f} Ha'.format(n, energy))
    
    if conv<= conv_tol:
        break
        
print()
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('final value of the ground-state energy = {:.8f} Ha'.format(energy))
print('Accuracy with respect to the FCI energy: {:.8f} Ha ({:.8f} kcal/mol)'.format(
    np.abs(energy - (- 1.136189454088)), np.abs(energy - (- 1.136189454088))*627.503
    )
)
print()

print('Final circuit parameters = \n', params)

iteration = 0, Energy = -0.88179557 Ha
iteration = 20, Energy = -1.13380513 Ha
iteration = 40, Energy = -1.13558756 Ha
iteration = 60, Energy = -1.13585794 Ha
iteration = 80, Energy = -1.13600617 Ha
iteration = 100, Energy = -1.13608848 Ha
iteration = 120, Energy = -1.13613394 Ha

Final convergence parameter = 0.00000099 Ha
final value of the ground-state energy = -1.13615709 Ha
Accuracy with respect to the FCI energy: 0.00003237 Ha (0.02031093 kcal/mol)

Final circuit parameters = 
 [[ 5.54193389e+00  1.30219523e-08  3.07479606e+00]
 [ 7.03997361e+00  6.28318530e+00 -3.07020901e+00]
 [ 2.98479079e+00 -2.09540998e-01 -4.16893297e-02]
 [ 1.28993324e+00  1.30907639e-12  4.56873497e+00]]
